In [ ]:
# from belle2_enhanced_framework import create_belle2_framework
# framework = create_belle2_framework(particle_type='vpho', memory_budget_gb=32.0,energy_condition='5S_scan')
# processes = framework.load_particle_data(
#     "/gpfs/group/belle2/users2022/kyldem/photoneff_updated/parquet_storage/try5",
#     particle='vpho'  # Automatically uses VPHO_KEYS columns
# )

In [ ]:
# !pip3 install polars
# !pip install dill
from belle2_enhanced_framework import create_belle2_framework, ParticleDataLoaderV3
framework = create_belle2_framework(energy_condition='4S_offres')
loader = ParticleDataLoaderV3(particle_type='vpho', config=framework.config)
processes = loader.load_flat_structure(
    base_dir='/pnfs/desy.de/belle/local/user/sraiz/photon_efficiency/gammaEffKLM_Prompt_combinedFilesMerged',
    data_type='selected'  # or 'matched'
    
)


    ═══════════════════════════════════════════════════════════════
                        Belle II Layer 2
                   Compute-First Data Structures
    ═══════════════════════════════════════════════════════════════
    
    Components:
    ├── UnifiedLazyDataFrame
    │   └── Manifests compute graphs as DataFrames
    ├── OptimizedUltraLazyDict
    │   └── Process-aware container with groups
    ├── MaterializationController
    │   └── Intelligent format selection
    ├── GraphOptimizationEngine
    │   └── Automatic compute graph optimization
    └── MemoryAwareExecutor
        └── Adaptive execution with spilling
    
    Key Features:
    • Zero-copy operations through compute graphs
    • Automatic billion-row handling
    • C++ acceleration for critical operations
    • Physics-specific optimizations
    • Full transformation tracking
    
    Usage:
    >>> framework = Belle2Layer2Framework(memory_budget_gb=16.0)
    >>> data = framework.load_processes("/data/belle2/

In [ ]:
!pip install psutil
from importlib import reload
import belle2_enhanced_framework
reload(belle2_enhanced_framework)
from belle2_enhanced_framework import create_belle2_framework, OptimizedUltraLazyDict
# Run analysis for each variable
# Define variables with their binning
variables = [
    ('pRecoil', 100, (0.1, 6)),

    # ('mu1P', 60, (0, 3)),
    # ('mu2P', 60, (0, 3)),
]
# Define physics cuts
full_cut = 'mu1nCDCHits>4&mu2nCDCHits>4&0.8>mu1clusterEoP&0.8>mu2clusterEoP&2.6179938779914944>pRecoilTheta>0.29670597283903605&11>totalMuonMomentum&absdPhi>1.5707963267948966&2.03>mu1Theta>0.61&2.03>mu2Theta>0.61&(absdPhiMu1>0.4014257279586958|absdThetaMu1>0.4014257279586958)&(absdPhiMu2>0.4014257279586958|absdThetaMu2>0.4014257279586958)&0.35>mu1clusterE&0.35>mu2clusterE&3>abs(m2Recoil)&min_deltaMuPRecoil>-0.01'
user_cuts = full_cut.split('&') if full_cut else []
all_results = {}
for var_name, bins, range_tuple in variables:
    print(f"\n📊 Analyzing {var_name}...")
    results = framework.run_progressive_analysis(
        processes,
        variable=var_name,
        bins=bins,
        range=range_tuple,
        cuts=user_cuts,
        stages=['baseline', 'candidates', 'cuts'],  # Standard progression
    )
    all_results[var_name] = results

In [4]:
results

{'variable': 'pRecoil',
 'configuration': {'bins': 100,
  'range': (0.1, 6),
  'cuts': ['mu1nCDCHits>4',
   'mu2nCDCHits>4',
   '0.8>mu1clusterEoP',
   '0.8>mu2clusterEoP',
   '2.6179938779914944>pRecoilTheta>0.29670597283903605',
   '11>totalMuonMomentum',
   'absdPhi>1.5707963267948966',
   '2.03>mu1Theta>0.61',
   '2.03>mu2Theta>0.61',
   '(absdPhiMu1>0.4014257279586958|absdThetaMu1>0.4014257279586958)',
   '(absdPhiMu2>0.4014257279586958|absdThetaMu2>0.4014257279586958)',
   '0.35>mu1clusterE',
   '0.35>mu2clusterE',
   '3>abs(m2Recoil)',
   'min_deltaMuPRecoil>-0.01'],
  'stages': ['baseline', 'candidates', 'cuts'],
  'energy_condition': '4S_offres',
  'luminosity_weighting': True},
 'histograms': {'baseline': {'hhISR-offprompt': (array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0.